In [ ]:
import pandas as pd

from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
# Add credentials file name/path here.
client = bigquery.Client.from_service_account_json('')

In [ ]:
query = ('SELECT * '
         'FROM (SELECT `by` as user_id, COUNT(*) AS num_comments '
         'FROM `bigquery-public-data.hacker_news.comments` '
         'WHERE `by` IS NOT NULL '
         'AND EXTRACT(YEAR FROM time_ts) = 2015 '
         'AND EXTRACT(MONTH FROM time_ts) = 10 '
         'AND EXTRACT(DAY FROM time_ts) < 8 '
         'GROUP BY `by` '
         'ORDER BY num_comments DESC '
         'LIMIT 400) users '
         'INNER JOIN `bigquery-public-data.hacker_news.comments` comments '
         'ON user_id = `by` '
         'WHERE EXTRACT(YEAR FROM time_ts) = 2015 '
         'AND EXTRACT(MONTH FROM time_ts) = 10 '
         'AND EXTRACT(DAY FROM time_ts) < 8;'
        )

In [ ]:
query_job = client.query(query)

iterator = query_job.result(timeout=120)
rows = list(iterator)

In [ ]:
# Transform the rows into a nice pandas dataframe.
df = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
df = df.drop(['user_id', 'num_comments'], axis=1)

# Look at the most prolific 10 commenters.
df.head(10)

In [ ]:
df.shape

In [ ]:
df.to_csv('first_week_oct_2015_comments_by_top_400.csv', index=False)